In [ ]:
#default_exp asyncUtil

# async
> tools to help writing async python codes

In [ ]:
#hide
from nbdev.showdoc import *

# async wrap

In [ ]:
#export
import asyncio
from functools import wraps, partial

def async_wrap(func):
    @wraps(func)
    async def run(*args, loop=None, executor=None, **kwargs):
        if loop is None:
            loop = asyncio.get_event_loop()
        pfunc = partial(func, *args, **kwargs)
        return await loop.run_in_executor(executor, pfunc)
    return run


In [ ]:
%%time
@async_wrap
def aSlowFunc(input_:str):
  time.sleep(2)
  return input_

## async func execute
import nest_asyncio, time
nest_asyncio.apply()
async def runASlowFunc(input_):
  return await aSlowFunc(input_)
async def runLoop():
  rtup = (runASlowFunc(i) for i in range (10))
  r = await asyncio.gather(*rtup)
  return r
asyncio.run(runLoop())

CPU times: user 5.13 ms, sys: 69 µs, total: 5.2 ms
Wall time: 4.01 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# thread mapping

In [ ]:
#export
import multiprocessing.dummy
from typing import Callable, List, Any, Iterable
from beartype import beartype
def asyncMap(f:Callable, data:Iterable[Any], threads:int = 5):
  p = multiprocessing.dummy.Pool(threads)
  return p.map(f,data)

In [ ]:
%%time
import time
asyncMap(lambda x: (x+1, time.sleep(1))[0] , range(10), threads = 100)

CPU times: user 14.1 ms, sys: 9.13 ms, total: 23.2 ms
Wall time: 1.02 s


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]